# Dual-Task

## DT | Configuration

In [ ]:
# Configuration
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

In [ ]:
cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

## DT | transforms

In [ ]:
import torchvision

image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]

nia_train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

nia_valid_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

aihub_train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

aihub_valid_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

aihub_test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

## DT | AIHub DataFrame

In [ ]:
from fpt.path import DTFR
from fpt.data import join_face_df

face_df = join_face_df(DTFR, "aihub_family")

## DT | Dataset

In [ ]:
import os
from easydict import EasyDict as edict
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from facenet.datasets.AIHubDataset import AIHubDataset
from nia_age.data import NiaDataset as nia

RANGE_TO_MEDIAN = nia.RANGE_TO_MEDIAN
AGE_GROUPS = nia.AGE_GROUPS
GROUP_TO_INDEX = {group: index for index, group in enumerate(AGE_GROUPS)}
TRAIN_FID_TO_INDEX = {idx: f"F{i:04d}" for idx, i in enumerate(range(1, 701))}
age_to_age_groups = nia.age_to_age_groups


class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, face_df, transform):
        super(FaceAgeDataset, self).__init__()
        self.face_dataset = ImageFolder(root=root_dir, transform=transform)
        self.face_df = face_df
        self.class_to_idx = self.face_dataset.class_to_idx
        self.samples = self.face_dataset.samples
        uuids = [
            img_path.rsplit(".", 1)[0].rsplit("/", 1)[1] for img_path, _ in self.samples
        ]
        unique_family_id = self.face_df.loc[uuids].family_id.unique()
        self.FID_TO_INDEX = {
            id: index for index, id in enumerate(sorted(unique_family_id))
        }

    def __len__(self):
        return len(self.face_dataset)

    def __getitem__(self, index):
        image, face_label = self.face_dataset[index]
        path, _ = self.face_dataset.samples[index]
        *_, key = os.path.splitext(path)[0].split("/")
        row = face_df.loc[key]
        sample = edict(
            {
                "image": image,
                "age": row.age,
                "age_class": GROUP_TO_INDEX[row.age_group],
                "file": path,
                "data_type": row.category,
                "family_id": row.family_id,
                "family_class": self.FID_TO_INDEX[row.family_id],
                "personal_id": row.target,
                "face_label": face_label,
                "key": key,
            }
        )
        return sample

In [ ]:
face_age_train_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/train_aihub_family",
    face_df=face_df,
    transform=aihub_train_transforms,
)

face_age_valid_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/valid_aihub_family",
    face_df=face_df,
    transform=aihub_valid_transforms,
)

face_age_test_dataset = FaceAgeDataset(
    root_dir="/home/jupyter/data/face-image/test_aihub_family",
    face_df=face_df,
    transform=aihub_test_transforms,
)

aihub_pairs_valid_dataset = AIHubDataset(
    dir="/home/jupyter/data/face-image/valid_aihub_family",
    pairs_path="/home/jupyter/data/pairs/valid/pairs_Age.txt",
    transform=aihub_valid_transforms,
)

aihub_pairs_test_dataset = AIHubDataset(
    dir="/home/jupyter/data/face-image/test_aihub_family",
    pairs_path="/home/jupyter/data/pairs/test/pairs_Age.txt",
    transform=aihub_valid_transforms,
)

## DT | DataLoader

In [ ]:
from torch.utils.data import DataLoader

train_batch_size = 32
valid_batch_size = 1
test_batch_size = 1

face_age_train_loader = DataLoader(
    face_age_train_dataset,
    batch_size=train_batch_size,
    num_workers=0,
    shuffle=True,
)

face_age_valid_loader = DataLoader(
    face_age_valid_dataset,
    batch_size=valid_batch_size,
    num_workers=0,
    shuffle=False,
)

face_age_test_loader = DataLoader(
    face_age_test_dataset,
    batch_size=test_batch_size,
    num_workers=0,
    shuffle=False,
)

aihub_pairs_valid_loader = DataLoader(
    aihub_pairs_valid_dataset,
    batch_size=valid_batch_size,
    num_workers=0,
    shuffle=False,
)

aihub_pairs_test_loader = DataLoader(
    aihub_pairs_test_dataset,
    batch_size=test_batch_size,
    num_workers=0,
    shuffle=False,
)

## DT | Loss function

In [ ]:
from typing import Callable
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import normalize, linear
from arcface_torch.losses import CombinedMarginLoss, ArcFace, CosFace
from nia_age.mean_variance_loss import MeanVarianceLoss
from nia_age.main_ae import LAMBDA_1, LAMBDA_2, START_AGE, END_AGE


NUM_CLASSES = (
    len(face_age_train_dataset.class_to_idx) if face_age_train_dataset else 2154
)


class FaceRecogFC(torch.nn.Module):
    def __init__(
        self,
        margin_loss: Callable,
        embedding_size: int,
        num_classes: int,
    ):
        super(FaceRecogFC, self).__init__()
        self.cross_entropy = CrossEntropyLoss()
        self.embedding_size = embedding_size
        self.weight = torch.nn.Parameter(torch.normal(0, 0.01, (1, embedding_size)))
        self.margin_loss = margin_loss
        self.num_classes = num_classes

    def forward(
        self,
        embeddings: torch.Tensor,
        labels: torch.Tensor,
    ):
        # labels
        labels.squeeze_()
        labels = labels.long()
        labels = labels.view(-1, 1)

        # embeddings
        norm_embeddings = normalize(embeddings)

        # weight
        weight = torch.nn.Parameter(
            torch.normal(0, 0.01, (self.num_classes, 512))
        ).cuda()
        norm_weight_activated = normalize(weight)
        norm_weight_activated.shape

        # logits
        logits = linear(norm_embeddings, norm_weight_activated)
        logits = logits.clamp(-1, 1)

        # softmax
        softmax = self.margin_loss(logits, labels)

        # loss
        loss = self.cross_entropy(softmax, labels.flatten())

        return loss


margin_loss = CombinedMarginLoss(
    64,
    *cfg.margin_list,
    cfg.interclass_filtering_threshold,
)

face_recog_fc = FaceRecogFC(
    margin_loss,
    512,
    NUM_CLASSES,
)

mean_variance_loss = MeanVarianceLoss(
    LAMBDA_1,
    LAMBDA_2,
    START_AGE,
    END_AGE,
)

In [ ]:
from torch import distributed
from arcface_torch.partial_fc_v2 import PartialFC_V2
from arcface_torch.losses import CombinedMarginLoss

if not distributed.is_initialized():
    try:
        rank = int(os.environ["RANK"])
        local_rank = int(os.environ["LOCAL_RANK"])
        world_size = int(os.environ["WORLD_SIZE"])
        distributed.init_process_group("nccl")
        
    except KeyError:
        rank = 0
        local_rank = 0
        world_size = 1
        distributed.init_process_group(
            backend="nccl",
            init_method="tcp://127.0.0.1:12584",
            rank=rank,
            world_size=world_size,
        )
        
margin_loss = CombinedMarginLoss(
    64,
    *cfg.margin_list,
    cfg.interclass_filtering_threshold,
)

module_partial_fc = PartialFC_V2(
    margin_loss,
    cfg.embedding_size,
    cfg.num_classes,
    cfg.sample_rate,
    cfg.fp16,
)

module_partial_fc.train().cuda()

In [ ]:
import numpy as np


def age_loss_func(age_pred, age_group_pred, sample, mean_variance_loss, criterion):
    dta = np.array(sample.data_type)
    age_sample_indices = dta != "Age"
    age_pred = age_pred[age_sample_indices]
    labels = sample.age[age_sample_indices].cuda()

    mean_loss, variance_loss = mean_variance_loss(age_pred, labels)
    age_softmax_loss = criterion(age_pred, labels)
    mean_loss, variance_loss, age_softmax_loss

    age_group_pred = age_group_pred[~age_sample_indices]
    age_group_labels = sample.age_class[~age_sample_indices].cuda()
    age_group_softmax_loss = criterion(age_group_pred, age_group_labels)
    return age_softmax_loss, age_group_softmax_loss

In [ ]:
import numpy as np


def kinship_loss_func(kinship_pred, sample, criterion):
    labels = sample.family_class.cuda()
    kinship_softmax_loss = criterion(kinship_pred, labels)
    return kinship_softmax_loss

## DT | Model

In [ ]:
import torch
from torch import nn
from arcface_torch.backbones import get_model
from nia_age.main_ae import AgeModel
from nia_age.main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS

network = "r50"
NUM_AGES = END_AGE - START_AGE + 1

face_age_model = get_model(network, dropout=0.0)
face_age_path = f"/home/jupyter/family-photo-tree/utils/model/arcface/{network}/backbone.pth"
face_age_model.load_state_dict(torch.load(face_age_path))

nia_age_model = AgeModel(NUM_AGES, NUM_AGE_GROUPS)
nia_age_path = "/home/jongphago/nia_age/result_model/model_0"
nia_age_model.load_state_dict(torch.load(nia_age_path))

In [ ]:
class AgeModule(nn.Module):
    def __init__(self, num_ages, num_age_groups):
        super(AgeModule, self).__init__()
        self.age_classifier = nn.Linear(512, num_ages)
        self.age_group_classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, num_age_groups),
        )

    def forward(self, x):
        age_pred = self.age_classifier(x)
        age_group_pred = self.age_group_classifier(x)
        return age_pred, age_group_pred


age_module = AgeModule(NUM_AGES, NUM_AGE_GROUPS)
saved_params = torch.load(nia_age_path)
selected_params = {
    k: v
    for k, v in saved_params.items()
    if "age_classifier" in k or "age_group_classifier" in k
}
age_module.load_state_dict(selected_params)

In [ ]:
class KinshipModule(nn.Module):
    def __init__(self, num_family_id):
        super(KinshipModule, self).__init__()
        self.kinship_classifier = nn.Linear(512, num_family_id)

    def forward(self, x):
        kinship_pred = self.kinship_classifier(x)
        return kinship_pred


NUM_TRAIN_FAMILY = 700
kinship_module = KinshipModule(NUM_TRAIN_FAMILY)

## DT | Optimizer

In [ ]:
from torch.optim import SGD
from arcface_torch.lr_scheduler import PolyScheduler

In [ ]:
momentum = 0.9  #
weight_decay = 5e-4  #
lr = 0.02

face_age_optimizer = SGD(
    params=[
        {"params": face_age_model.parameters()},
        {"params": module_partial_fc.parameters()},
        {"params": kinship_module.parameters()}
    ],
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay,
)

In [ ]:
num_epoch = 2
world_size = 1
cfg.total_batch_size = cfg.batch_size * world_size
cfg.warmup_step = cfg.num_image // cfg.total_batch_size * cfg.warmup_epoch
cfg.total_step = cfg.num_image // cfg.total_batch_size * num_epoch

lr_scheduler = PolyScheduler(
    optimizer=face_age_optimizer,
    base_lr=lr,
    max_steps=cfg.total_step,  # 1452
    warmup_steps=cfg.warmup_step,
    last_epoch=-1,
)

## DT | Train

In [ ]:
def tensor_to_int(x):
    return x.cpu().data.numpy().item()

In [ ]:
face_age_model.train()
face_age_model.cuda()
age_module.train()
age_module.cuda()
kinship_module.train()
kinship_module.cuda()
module_partial_fc.train()
module_partial_fc.cuda()
mean_variance_loss.train()
mean_variance_loss.cuda()
cross_entropy_loss = CrossEntropyLoss().cuda()

In [ ]:
for _, sample in enumerate(face_age_train_loader):
    embeddings = face_age_model(sample.image.cuda())

    age_pred, age_group_pred = age_module(embeddings)

    fr_loss: torch.Tensor = module_partial_fc(embeddings, sample.face_label.cuda())
    age_loss, age_group_loss = age_loss_func(
        age_pred, age_group_pred, sample, mean_variance_loss, cross_entropy_loss
    )
    kinship_pred = kinship_module(embeddings)
    kinship_loss = kinship_loss_func(kinship_pred, sample, cross_entropy_loss)
    loss = fr_loss + age_loss + age_group_loss + kinship_loss
    if _ % 10 == 0:
        print(
            f"{_:4d},\
            loss: {tensor_to_int(loss):8.4f},\
            fr: {tensor_to_int(fr_loss):4.2f},\
            age: {tensor_to_int(age_loss):4.2f},\
            age_group: {tensor_to_int(age_group_loss):4.2f}\
            kinship: {tensor_to_int(kinship_loss):4.2f}"
        )

    torch.nn.utils.clip_grad_norm_(face_age_model.parameters(), 5)
    face_age_optimizer.zero_grad()
    loss.backward()
    face_age_optimizer.step()
    lr_scheduler.step()
    break